In [2]:
# !pip install pinecone

In [20]:
import os
from dotenv import load_dotenv
from pathlib import Path
from pinecone import Pinecone, ServerlessSpec

dotenv_path = Path('/Users/sanchaynibagade/Documents/github/grant_matching/env/llm.env')
load_dotenv(dotenv_path=dotenv_path)
MY_KEY = os.getenv('PINECONE_KEY')


In [4]:
pc = Pinecone(api_key=MY_KEY)

In [ ]:
# # To get the unique host for an index, 
# # see https://docs.pinecone.io/guides/data/target-an-index
# index = pc.Index(host="https://testembedding-jk789yn.svc.aped-4627-b74a.pinecone.io")

# index.upsert(
#   vectors=[
#     {
#       "id": "A", 
#       "values": [0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1], 
#       "metadata": {"genre": "comedy", "year": 2020}
#     },
#     {
#       "id": "B", 
#       "values": [0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2],
#       "metadata": {"genre": "documentary", "year": 2019}
#     },
#     {
#       "id": "C", 
#       "values": [0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3],
#       "metadata": {"genre": "comedy", "year": 2019}
#     },
#     {
#       "id": "D", 
#       "values": [0.4, 0.4, 0.4, 0.4, 0.4, 0.4, 0.4, 0.4],
#       "metadata": {"genre": "drama"}
#     }
#   ],
#   namespace="test-namespace"
# )

{'upserted_count': 4}

## Generating Embeddings

In [ ]:
from langchain_ollama import OllamaEmbeddings
import pandas as pd

In [ ]:
df = pd.read_csv("../../data/realistic_restaurant_reviews.csv")
embeddings = OllamaEmbeddings(model="mxbai-embed-large")

In [23]:
pc = Pinecone(api_key=MY_KEY)
index = pc.Index(host="https://restuarantdata-jk789yn.svc.aped-4627-b74a.pinecone.io")

In [43]:
vectors = []

for i, row in df.iterrows():
    text_to_embedd = row["Title"] + " " + row["Review"]
    metadata= {"rating": row["Rating"], "date": row["Date"], "chunk_text": text_to_embedd}
    id=str(i)
    
    embedds = embeddings.embed_query(text_to_embedd)

    vec = {
             "id": id,
             "values": embedds,
             "metadata": metadata
            }
    vectors.append(vec)


In [44]:
index.upsert(vectors=vectors,
             namespace="restuarant-data"
             )

{'upserted_count': 123}

## Semantic Search on the embeddings

In [76]:
results = index.search(
    namespace="restuarant-data",
    query={
        "inputs": {"text": "How is the ambience of the restuarant?"}, 
        "top_k": 5
    },
    fields=["chunk_text","Rating"]
)

In [77]:
results['result']['hits']

[{'_id': '65',
  '_score': 0.042206134647130966,
  'fields': {'chunk_text': 'Too much garlic I love garlic but this was '
                           'excessive - felt like they mistook cloves for '
                           'teaspoons in the recipe. My breath was lethal '
                           'afterward, and the powerful flavor overwhelmed the '
                           'other ingredients.'}},
 {'_id': '79',
  '_score': 0.041884519159793854,
  'fields': {'chunk_text': 'Lackluster veggie options Their vegetable toppings '
                           'lacked freshness and flavor - the mushrooms were '
                           'clearly canned, the peppers were soft and faded, '
                           'and the onions were sparse. Disappointing for a '
                           'place that advertises farm-fresh ingredients.'}},
 {'_id': '49',
  '_score': 0.037499189376831055,
  'fields': {'chunk_text': 'Refrigerated dough taste The crust had that '
                           

In [73]:
query_text = "How is the ambience of the restuarant?"
query_embedds = embeddings.embed_query(query_text)
results = index.query(
    namespace="restuarant-data",
    vector=query_embedds,
    include_metadata=True,
    top_k=3
    
)

In [74]:
results['matches']

[{'id': '8',
  'metadata': {'chunk_text': 'Mediocre at best Nothing terrible but nothing '
                             'special either. The crust was okay, toppings were '
                             "standard, and service was fine. It's the kind of "
                             "place you go when you're in the area, but "
                             "wouldn't make a special trip for.",
               'date': '2024-03-05',
               'rating': 3.0},
  'score': 0.63597697,
  'values': []},
 {'id': '18',
  'metadata': {'chunk_text': 'Too noisy for conversation The pizza was good - I '
                             'especially liked their sausage and mushroom '
                             'combination - but the restaurant is like an echo '
                             "chamber. Couldn't hear my date across the table. "
                             'Better as a takeout option.',
               'date': '2024-01-30',
               'rating': 3.0},
  'score': 0.624991775,
  'values': 